In [111]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15962041257352351903
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16591054485076167791
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4424036384668240335
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11276946637
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15242822112926518825
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [29]:
# Imports
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import time
from math import log10, floor

In [30]:
# Define variables
n_vars = 3
var_ids = list(range(n_vars))
var_names = ['var' + str(i) for i in var_ids]
var_weights = [0.1, 0.6, 0.3] # variable distribution of mock data
n_time_steps = 6
n_individuals = 1000

In [31]:
# Helper function(s)

# round a number to n significant digits
def round_to_n(x, n = 2):
    return round(x, -int(floor(log10(abs(x)))) + (n - 1)) if x != 0 else 0

# visualize the output of the generator
def visualize_output(generator, z, n = 2):
    p = np.reshape(generator.predict(z), (n_time_steps, n_vars))
    p.shape
    for t in range(p.shape[0]):
        tmp = []
        for f in range(p.shape[1]):
            tmp.append(round_to_n(p[t,f], n))
        print(tmp)

In [32]:
# Generate mock data

events = []

start_time = time.time()

for indv in range(n_individuals):
    tmp = []
    for t in range(np.random.randint(n_time_steps // 2, n_time_steps + 1)):
        var = np.random.choice(var_names, p=var_weights)
        tmp.append(var)
    events.append(tmp)
        
print('time taken:', round_to_n(time.time() - start_time), 'seconds')

for i in range(10):
    print(events[i])

time taken: 0.13 seconds
['var1', 'var0', 'var1', 'var1', 'var2', 'var1']
['var1', 'var1', 'var2']
['var1', 'var0', 'var1']
['var0', 'var2', 'var2', 'var2', 'var1']
['var1', 'var2', 'var1', 'var2', 'var1', 'var1']
['var1', 'var1', 'var1', 'var1', 'var1']
['var1', 'var1', 'var0', 'var2', 'var1']
['var2', 'var1', 'var1', 'var1', 'var1', 'var1']
['var1', 'var1', 'var2', 'var1', 'var1']
['var1', 'var0', 'var2', 'var0']


In [33]:
# Transform the data into a format required by the Embedding layer

t = Tokenizer()
t.fit_on_texts(events)
vocab_size = len(t.word_index) + 1
encoded_data = t.texts_to_sequences(events)
data = pad_sequences(encoded_data, maxlen=n_time_steps, padding='post')
data[:10, :]

array([[1, 3, 1, 1, 2, 1],
       [1, 1, 2, 0, 0, 0],
       [1, 3, 1, 0, 0, 0],
       [3, 2, 2, 2, 1, 0],
       [1, 2, 1, 2, 1, 1],
       [1, 1, 1, 1, 1, 0],
       [1, 1, 3, 2, 1, 0],
       [2, 1, 1, 1, 1, 1],
       [1, 1, 2, 1, 1, 0],
       [1, 3, 2, 3, 0, 0]], dtype=int32)

In [34]:
# Divide the data into train and test sets 

data_train, data_test = train_test_split(data, test_size = 0.2)
print(data_train.shape, data_test.shape)

(800, 6) (200, 6)


In [37]:
# Define the discriminator

discriminator = Sequential(name = 'discriminator')
discriminator.add(Embedding(vocab_size, 3, input_length=n_time_steps))
discriminator.add(LSTM(5))
discriminator.add(Dense(1, activation='sigmoid'))
#discriminator = Model(inp_d, out_d, name='discriminator')
discriminator.summary()


Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 6, 3)              12        
_________________________________________________________________
lstm_2 (LSTM)                (None, 5)                 180       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6         
Total params: 198
Trainable params: 198
Non-trainable params: 0
_________________________________________________________________


In [40]:
x = data_train[0]
discriminator.predict(np.reshape(x, (1, -1)))

array([[0.50223106]], dtype=float32)

In [42]:
# Define the generator
# TODO: implement RelGAN to some extent

noise_length = 3

inp_g = Input(shape=(n_time_steps, noise_length))
lstm_g = LSTM(5, return_sequences=True)(inp_g)
out_g = Dense(3, activation='softmax')(lstm_g)
generator = Model(inp_g, out_g, name='generator')
generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 6, 3)]            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 6, 5)              180       
_________________________________________________________________
dense_3 (Dense)              (None, 6, 3)              18        
Total params: 198
Trainable params: 198
Non-trainable params: 0
_________________________________________________________________


In [43]:
z = np.random.normal(scale = 0.5, size = (1, n_time_steps, noise_length))
generator.predict(z).shape

(1, 6, 3)

In [140]:
# Define the trainable discriminator model
discriminator.trainable = True

discriminator_model = Sequential()
discriminator_model.add(discriminator)
discriminator_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
discriminator_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
discriminator (Model)        (None, 1)                 186       
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________


In [141]:
# Define the trainable adversarial model
discriminator.trainable = False

adversarial_model = Sequential()
adversarial_model.add(generator)
adversarial_model.add(discriminator)
adversarial_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
adversarial_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generator (Model)            (None, 20, 3)             673       
_________________________________________________________________
discriminator (Model)        (None, 1)                 186       
Total params: 859
Trainable params: 673
Non-trainable params: 186
_________________________________________________________________


In [142]:
# Define a train function

def train(batch_size = 50, epochs = 10, print_step = 1):
    for epoch in range(epochs):
        idx_true = np.random.choice(data_train.shape[0], size = batch_size, replace = False)
        x_true = data_train[idx_true, :, :]
        z = np.random.normal(scale = 1, size = (batch_size, n_time_steps, noise_length))
        
        x_fake = generator.predict(z)
        
        x = np.concatenate((x_true, x_fake))
        y = np.ones([2 * batch_size, 1])
        y[batch_size:, :] = 0
        discriminator.trainable = True
        d_loss = discriminator_model.train_on_batch(x, y)
        
        y = np.ones([batch_size, 1])
        discriminator.trainable = False
        z = np.random.normal(scale = 1, size = (batch_size, n_time_steps, noise_length)) 
        a_loss = adversarial_model.train_on_batch(z, y)
        
        if (epoch % print_step) == 0:
            log_mesg = "%d: [D loss: %f, acc: %f]" % (epoch, d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
            print(log_mesg)
        

In [143]:
# run training function

start_time = time.time()
train(batch_size = 100, epochs = 500, print_step = 50)
print('time taken:', round_to_n(time.time() - start_time), 'seconds')

0: [D loss: 0.715298, acc: 0.500000]  [A loss: 0.816451, acc: 0.000000]
50: [D loss: 0.663767, acc: 0.520000]  [A loss: 0.812898, acc: 0.000000]
100: [D loss: 0.727041, acc: 0.325000]  [A loss: 0.614322, acc: 1.000000]
150: [D loss: 0.623657, acc: 0.890000]  [A loss: 0.790276, acc: 0.000000]
200: [D loss: 0.842737, acc: 0.265000]  [A loss: 0.549213, acc: 0.780000]
250: [D loss: 0.737081, acc: 0.500000]  [A loss: 0.819864, acc: 0.000000]
300: [D loss: 0.505884, acc: 0.980000]  [A loss: 1.137674, acc: 0.000000]
350: [D loss: 0.501969, acc: 0.825000]  [A loss: 1.086113, acc: 0.260000]
400: [D loss: 0.662116, acc: 0.490000]  [A loss: 0.555154, acc: 1.000000]
450: [D loss: 0.540381, acc: 0.985000]  [A loss: 0.792217, acc: 0.010000]
time taken: 150.0 seconds


In [144]:
z = np.random.normal(scale = 0.5, size = (1, n_time_steps, noise_length))
visualize_output(generator, z, 1) 

[0.2, 0.5, 0.3]
[0.2, 0.6, 0.2]
[0.2, 0.5, 0.3]
[0.2, 0.6, 0.3]
[0.1, 0.6, 0.3]
[0.1, 0.7, 0.2]
[0.08, 0.7, 0.2]
[0.07, 0.6, 0.3]
[0.07, 0.5, 0.5]
[0.05, 0.6, 0.3]
[0.03, 0.7, 0.2]
[0.03, 0.7, 0.3]
[0.03, 0.7, 0.3]
[0.02, 0.7, 0.3]
[0.03, 0.6, 0.4]
[0.02, 0.6, 0.3]
[0.02, 0.7, 0.3]
[0.03, 0.5, 0.5]
[0.03, 0.4, 0.5]
[0.03, 0.4, 0.5]
